In [1]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('categorized_data.csv')

print("데이터 로드")
display(df.head())

데이터 로드


,brand,name,price,event,img_url,category
0,7Eleven,HK)새싹보리500ml,2500,1+1,https://www.7-eleven.co.kr/upload/product/8809...,음료
1,7Eleven,LG)샤프란아우라1L(스윗만다린),12900,1+1,https://www.7-eleven.co.kr/upload/product/8801...,생활/위생용품
2,7Eleven,LG)샤프란아우라1L(매그놀리아),12900,1+1,https://www.7-eleven.co.kr/upload/product/8801...,생활/위생용품
3,7Eleven,아모레)미장센퍼펙트샴푸680ml_H,18000,1+1,https://www.7-eleven.co.kr/upload/product/8809...,생활/위생용품
4,7Eleven,아모레)미장센퍼펙트린스680ml_H,18000,1+1,https://www.7-eleven.co.kr/upload/product/8809...,생활/위생용품


카테고리별 가격 분포

In [5]:
fig_box = px.box(df, 
                 x='category', 
                 y='price', 
                 color='category',
                 title='💰 카테고리별 가격대 분포',
                 points="all")

fig_box.show()